In [2]:
import json
import pandas as pd
import re

# Intro

Contains two colums
* comment - actual comments
* isScam - whether its a scam or not

## bad_final.json

In [2]:
badFinalName = "./bad_final.json"

In [3]:
Lines = open(badFinalName, 'r').readlines()
data = []
for itr,line in enumerate(Lines):
    record = json.loads(line)
    comment = record.get("comments")
    data.append([' '.join(comment), "FALSE"])
dfBadFinal = pd.DataFrame(data,columns=["Statement","Label"])

## good (1).txt

In [4]:
goodName = "./good (1).txt"
regex = r'"(.*)"'

In [5]:
Lines = open(goodName, 'rb').read().decode()
splitLines = Lines.split("Comment")

In [6]:
data = []
for l in splitLines:
    matches = re.finditer(regex, l, re.MULTILINE)
    data.append([' '.join([m[1] for m in matches]), "TRUE"])
dfGood1 = pd.DataFrame(data,columns=["Statement","Label"])  

## risky(1).txt

In [7]:
riskyName = "./risky (1).txt"
regex = r'"(.*)"'

In [8]:
Lines = open(riskyName, 'rb').read().decode()
splitLines = Lines.split("Comment")

In [9]:
data = []
for l in splitLines:
    matches = re.finditer(regex, l, re.MULTILINE)
    data.append([' '.join([m[1] for m in matches]), "FALSE"])
dfRisky1 = pd.DataFrame(data,columns=["Statement","Label"]) 

## Suspected.json

In [10]:
suspectedName = "./suspects.json"
regex = r'<description>([^<]*)</description>'

In [11]:
pdData = pd.read_json(suspectedName)
pdData = pdData.drop('wallet', axis=1)

In [14]:
pdData

,comments
0,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."
1,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."
2,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."
3,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."
4,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."
5,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."
6,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."
7,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."
8,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."
9,"<?xml version=""1.0"" encoding=""utf-8""?>\n<rss x..."


In [12]:
def processXML(data):
    matches = re.finditer(regex, data, re.MULTILINE)
    return ' '.join([m[1] for m in matches])

In [19]:
pdData["Statement"] = pdData["comments"].apply(processXML)
pdData["Label"] = "FALSE"
pdData["Statement"] = pdData["Statement"]
dfSuspected = pdData.drop("comments",axis=1)

## Results

In [20]:
dfFinal = dfBadFinal
dfFinal = dfFinal.append(dfGood1, ignore_index=True)
dfFinal = dfFinal.append(dfRisky1,ignore_index=True)
dfFinal = dfFinal.append(dfSuspected,ignore_index=True)
dfFinal = dfFinal[dfFinal["Statement"]!='']

In [27]:
dfFinal.to_pickle("./Combined.pkl")
dfFinal.to_csv("./Combined.csv", index=False)

In [26]:
dfFinal.describe()

,Statement,Label
count,445,445
unique,425,2
top,Scammerâ€™s wallet,FALSE
freq,7,435


## Process pkl

In [4]:
data = pd.read_pickle("./Combined.pkl")

In [5]:
data.head()

,Statement,Label
0,Wallet belongs to a scammer,FALSE
1,ok this is a scammer account&lt;br&gt;I ask yo...,FALSE
2,Contact cybershark591@gmail.com Contact the of...,FALSE
3,"SCAM ACCOUNT SCAMMER ACCOUNT, REPORT TO THE AU...",FALSE
4,SEND MONEY BACK. I NEED IT PLEASE 0.78 ETH 0xb...,FALSE


In [7]:
data.to_csv("./custom_processed.csv", index=False)